<a href="https://colab.research.google.com/github/bish-ai/Bishal.py/blob/main/house_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
pip install pandas scikit-learn fastapi uvicorn joblib


In [82]:
import pandas as pd
house_data=pd.read_csv("https://raw.githubusercontent.com/codebasics/py/master/DataScience/BangloreHomePrices/model/bengaluru_house_prices.csv")
house_data.head()
house_data.shape
house_data.info()
house_data.describe()
house_data.isnull().sum()
import pandas as pd
from sklearn.model_selection import train_test_split
house_data.drop(columns=["area_type","availability"], inplace=True)
house_data.isnull().sum()
x=house_data[["location","size","society"]]
y=house_data[["bath","balcony"]]
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.5)
from sklearn.impute import SimpleImputer
impute= SimpleImputer(strategy="most_frequent")
x_train_1=impute.fit_transform(x_train)
y_train_1=impute.fit_transform(y_train)

# Check for null values in the imputed arrays
print("Null values in x_train_1 after imputation:")
print(pd.DataFrame(x_train_1).isnull().sum())
print("\nNull values in y_train_1 after imputation:")
print(pd.DataFrame(y_train_1).isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     13320 non-null  object 
 1   availability  13320 non-null  object 
 2   location      13319 non-null  object 
 3   size          13304 non-null  object 
 4   society       7818 non-null   object 
 5   total_sqft    13320 non-null  object 
 6   bath          13247 non-null  float64
 7   balcony       12711 non-null  float64
 8   price         13320 non-null  float64
dtypes: float64(3), object(6)
memory usage: 936.7+ KB
Null values in x_train_1 after imputation:
0    0
1    0
2    0
dtype: int64

Null values in y_train_1 after imputation:
0    0
1    0
dtype: int64


In [108]:
x=house_data[["location","size"]]
y=house_data[["balcony"]]
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.5,test_size=0.5,random_state=43)
x.head()


,location,size
0,Electronic City Phase II,2 BHK
1,Chikka Tirupathi,4 Bedroom
2,Uttarahalli,3 BHK
3,Lingadheeranahalli,3 BHK
4,Kothanur,2 BHK


In [124]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import joblib

# Initialize and train the Linear Regression model using preprocessed data
lr_model = LinearRegression()
lr_model.fit(x_train_preprocessed, y_train_imputed)

# Make predictions on the preprocessed training data
y_train_pred = lr_model.predict(x_train_preprocessed)

# Calculate the R-squared score on the training data
r2_train = r2_score(y_train_imputed, y_train_pred)

print(f"R-squared score on the training data: {r2_train:.4f}")
joblib.dump(lr_model,"house_prediction.pkl")
print("hogya model saved ,i done it ")

R-squared score on the training data: 0.1065
hogya model saved ,i done it 


In [126]:
%%writefile app.py



Writing app.py


In [129]:
%%writefile app.py
import streamlit as st
import joblib
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import re

# Define the convert_sqft_to_num function as it was used in preprocessing
def convert_sqft_to_num(x):
    try:
        tokens = x.split('-')
        if len(tokens) == 2:
            return (float(tokens[0]) + float(tokens[1])) / 2
        try:
            return float(x)
        except ValueError:
            return None  # Or np.nan
    except AttributeError:
        return None # Or np.nan for non-string values

# Load the preprocessor and model
try:
    preprocessor = joblib.load('preprocessor.pkl')
    model = joblib.load('house_prediction.pkl')
except FileNotFoundError:
    st.error("Error: 'preprocessor.pkl' or 'house_prediction.pkl' not found.")
    st.stop()

st.title('Bangalore House Balcony Prediction')
st.write('Enter the details of the house to predict the number of balconies.')

# User inputs
location_options = preprocessor.named_transformers_['cat'].categories_[0].tolist()
size_options = preprocessor.named_transformers_['cat'].categories_[1].tolist()

location = st.selectbox('Location', sorted(location_options))
size = st.selectbox('Size', sorted(size_options))
total_sqft = st.text_input('Total Square Feet (e.g., 1200 or 1000-1200)')
bath = st.number_input('Number of Bathrooms', min_value=1, max_value=10, value=2)

if st.button('Predict Number of Balconies'):
    # Preprocess user input
    total_sqft_processed = convert_sqft_to_num(total_sqft)

    # Create a DataFrame for the input
    input_data = pd.DataFrame([[location, size, total_sqft_processed, bath]],
                              columns=['location', 'size', 'total_sqft', 'bath'])

    # Impute potential NaNs in total_sqft if convert_sqft_to_num returned None
    # Note: The preprocessor handles imputation for numerical features, so we ensure the input is ready for it.
    # For the 'total_sqft' column specifically, if `convert_sqft_to_num` returned None, it will be NaN
    # The ColumnTransformer's SimpleImputer for numerical features will handle it.

    try:
        processed_input = preprocessor.transform(input_data)
        prediction = model.predict(processed_input)
        # Round the prediction to the nearest integer as balcony count must be an integer
        predicted_balcony = round(prediction[0])
        st.success(f'Predicted Number of Balconies: {int(predicted_balcony)}')
    except Exception as e:
        st.error(f'An error occurred during prediction: {e}')

# Instructions on how to run the app
st.sidebar.header('How to Run This App')
st.sidebar.markdown(
    """1. Save the model ('house_prediction.pkl') and preprocessor ('preprocessor.pkl') in the same directory as `app.py`.
2. Open your terminal or command prompt.
3. Navigate to the directory where `app.py` is saved.
4. Run the command: `streamlit run app.py`
5. The application will open in your web browser.
    """
)


Overwriting app.py


In [130]:
import joblib

joblib.dump(preprocessor, 'preprocessor.pkl')
print("Preprocessor saved successfully as 'preprocessor.pkl'")

Preprocessor saved successfully as 'preprocessor.pkl'
